In [1]:
import os
import segmentation_models as sm
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

Segmentation Models: using `tf.keras` framework.


# save whole unet model architecture

In [6]:
def model_loading(file = '04_unet_training/best_model.h5', CLASSES = ['papaya'], img_size = 320):
    os.environ["SM_FRAMEWORK"] = "tf.keras"
    BACKBONE = 'resnet18'
    n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1) 
    activation = 'sigmoid' if n_classes == 1 else 'softmax'
    model = sm.Unet(BACKBONE, classes=n_classes, activation=activation, input_shape=(img_size, img_size*2, 3))
    model.load_weights(file) 
    return model
model_unet = model_loading()
model_unet.save('08_final_weight/unet_resnet18.h5')

# save whole classification model architecture

In [7]:
def model_build(activation_, normal_, l2_, pooling_, image_type):
    if image_type=='merge':
        base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    if image_type=='chanel3':
        base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    if image_type=='chanel3_127':
        base_model = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
    if image_type=='chanel6':
        base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 6))
    if image_type=='chanel12':
        base_model = MobileNetV2(weights=None, include_top=False, input_shape=(224, 224, 12))
    if image_type=='multi':
        base_model_1 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
        base_model_2 = MobileNetV2(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
        # 連接自訂層
        for layer in base_model_1.layers:
            layer._name = layer._name + str("_1")
        for w in base_model_1.weights:
            w._handle_name = w.name + str("_1")
        for layer in base_model_2.layers:
            layer._name = 'EP_' + layer._name + str("_2")
        for w in base_model_2.weights:
            w._handle_name = 'EP_' + w.name + str("_2")
        base_model = layers.concatenate([base_model_1.output, base_model_2.output])
    
    if pooling_ == "max":
        if image_type=='multi':
            x = layers.GlobalMaxPooling2D()(base_model)
        else:
            x = layers.GlobalMaxPooling2D()(base_model.output)
    if pooling_ == "ave":
        if image_type=='multi':
            x = layers.GlobalAveragePooling2D()(base_model)
        else:
            x = layers.GlobalAveragePooling2D()(base_model.output)
    if pooling_ == "conv":
        if image_type=='multi':
            x = layers.Conv2D(1280, (7, 7), activation="relu")(base_model)
        else:
            x = layers.Conv2D(1280, (7, 7), activation="relu")(base_model.output)
    if pooling_ == 'all':
        x1 = layers.GlobalMaxPooling2D()(base_model.output)
        x2 = layers.GlobalAveragePooling2D()(base_model.output)
        x3 = layers.Conv2D(1280, (7, 7), activation="relu")(base_model.output)
        x3 = layers.BatchNormalization()(x3)
        x3 = layers.Activation(activation_)(x3)
        x3 = layers.Flatten()(x3)
        x = layers.concatenate([x1, x2, x3])
    
    if pooling_ == "conv":
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        x = layers.Flatten()(x)
    else:        
        if normal_ == 'drop':
            x = layers.Dropout(0.25)(x)
        if normal_ == 'batch':
            x = layers.BatchNormalization()(x)
            x = layers.Activation(activation_)(x)
        
    x = layers.Dense(1024, activation=activation_, kernel_initializer='he_normal', kernel_regularizer=l2(float(l2_)))(x)    
    if normal_ == 'drop':
        x = layers.Dropout(0.25)(x)
    if normal_ == 'batch':
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        
    x = layers.Dense(256, activation=activation_, kernel_initializer='he_normal', kernel_regularizer=l2(float(l2_)))(x)
    if normal_ == 'drop':
        x = layers.Dropout(0.25)(x)
    if normal_ == 'batch':
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        
    x = layers.Dense(64, activation=activation_, kernel_initializer='he_normal', kernel_regularizer=l2(float(l2_)))(x)
    if normal_ == 'drop':
        x = layers.Dropout(0.25)(x)
    if normal_ == 'batch':
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation_)(x)
        
    x = layers.Dense(6, activation='softmax')(x)

    # 設定新模型的 inputs/outputs
    if image_type=='multi':
        model = Model(inputs=[base_model_1.input, base_model_2.input], outputs=x)
    else:
        model = Model(inputs=base_model.input, outputs=x)
        
    return model

h5_route = '07_classification_training_chanel3/sgd/ave/relu_batch_0.1/weights_acc.h5'
model_classification = model_build(activation_ = 'relu', normal_ = 'batch', l2_ = 0.1, 
                                   pooling_ = 'ave', image_type = 'chanel3')
model_classification.load_weights(h5_route)
model_classification.save('08_final_weight/classification_mobilenetv2.h5')